# Question 1 part 1

# Implementation of Decision tree, Logistic regression, KNN neighbour, SVM,Gradient Boosting and Neural nets on event dataset

In [18]:
#importing common packages for the techniques
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import scikitplot as sckplt
from sklearn.metrics import confusion_matrix, precision_recall_curve, average_precision_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import *
import warnings
warnings.filterwarnings('ignore')

In [73]:
#Technique specific packages
from sklearn import tree
from sklearn import neighbors, datasets
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [79]:
#Loading Data 
event=pd.read_excel("C:\\Users\\agraw\\Desktop\\Predictive\\Assignment 3\\HW3.xlsx")

In [4]:
#Exploring data
print(event.describe().transpose())

                       count         mean          std  min      25%  \
sequence_number       2000.0  1000.500000   577.494589  1.0   500.75   
US                    2000.0     0.824500     0.380489  0.0     1.00   
source_a              2000.0     0.126500     0.332495  0.0     0.00   
source_c              2000.0     0.056000     0.229979  0.0     0.00   
source_b              2000.0     0.060000     0.237546  0.0     0.00   
source_d              2000.0     0.041500     0.199493  0.0     0.00   
source_e              2000.0     0.151000     0.358138  0.0     0.00   
source_m              2000.0     0.016500     0.127420  0.0     0.00   
source_o              2000.0     0.033500     0.179983  0.0     0.00   
source_h              2000.0     0.052500     0.223089  0.0     0.00   
source_r              2000.0     0.068500     0.252665  0.0     0.00   
source_s              2000.0     0.047000     0.211692  0.0     0.00   
source_t              2000.0     0.021500     0.145080  0.0     

In [5]:
#Dimension of the data set
n_samples, n_features = event.shape
print ('The dimensions of the data set are', n_samples, 'by', n_features)

The dimensions of the data set are 2000 by 25


In [80]:
#split dataset in features and target variable
X=event.copy(deep=True)
columns = ['sequence_number', 'Purchase', 'Spending']
X.drop(columns, inplace=True, axis=1)
y = event.loc[:,'Spending'] # Target variable

#Test_train_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=42)

In [81]:
#Pre-processing - scaling of data 
scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(X_train)
x_train = pd.DataFrame(x_train_scaled)

x_test_scaled = scaler.fit_transform(X_test)
x_test = pd.DataFrame(x_test_scaled)

## Decision Tree

In [9]:
# Optimize the Decision tree  by hyperparameter tuning using cross validation
# Defining different parameters for tuning the model
d_range = list(range(1,20))
c_options = ["mse", "mae"]
samples_split=list(np.arange(0.05, 0.5, 0.05))
leaf_nodes=[10]

param_grid = dict(max_depth = d_range, criterion = c_options,min_samples_split=samples_split,max_leaf_nodes=leaf_nodes)
#print (param_grid)
model = tree.DecisionTreeRegressor()

grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19],
           

### Final reports for the Decision tree 

In [15]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-19588.52574255357
{'criterion': 'mae', 'max_depth': 6, 'max_leaf_nodes': 10, 'min_samples_split': 0.05}
DecisionTreeRegressor(criterion='mae', max_depth=6, max_features=None,
                      max_leaf_nodes=10, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.05, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
mean square error 19819.01122275
root mean square error 140.78001002539386
mean absolute error 70.26956666666665


## Linear regression 

In [20]:
# Different parameters for tuning the model
param_grid = {}
#print (param_grid)
model = LinearRegression()

#mean_squared_error
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None, param_grid={}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

### Final reports for linear regression

In [21]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-16281.288936134777
{}
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
mean square error 17597.142998353287
root mean square error 132.65422344710058
mean absolute error 84.60580802030539


## Lasso regression 

In [59]:
# Different parameters for tuning the model
#Linear model
param_grid = {'alpha':list(np.arange(0.1,2,0.1))}
model=linear_model.Lasso()

#mean_squared_error
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6,
                                   0.7000000000000001, 0.8, 0.9, 1.0, 1.1,
                                   1.2000000000000002, 1.3000000000000003,
                                   1.4000000000000001, 1.5000000000000002, 1.6,
                                   1.7000000000000002, 1.8000000000000003,
                                   1.9000000000000001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

### Final reports for lasso regression

In [60]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-16193.861459323778
{'alpha': 0.4}
Lasso(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
mean square error 17352.65854388237
root mean square error 131.7294900312089
mean absolute error 84.33790135336872


## Ridge regression 

In [61]:
# Different parameters for tuning the model
param_grid = {'alpha':list(np.arange(0.1,2,0.1))}
model=linear_model.Ridge()
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6,
                                   0.7000000000000001, 0.8, 0.9, 1.0, 1.1,
                                   1.2000000000000002, 1.3000000000000003,
                                   1.4000000000000001, 1.5000000000000002, 1.6,
                                   1.7000000000000002, 1.8000000000000003,
                                   1.9000000000000001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

### Final reports for Ridge regression

In [62]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-16281.88861859038
{'alpha': 0.1}
Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
mean square error 17535.842010883007
root mean square error 132.42296632715568
mean absolute error 84.279112555381


## KNN 

In [22]:
# Optimize the KNN model by hypertuning using cross validation
# Different parameters for tuning the model
n_range = list(range(1,30))
w_options = ["uniform", "distance"]

param_grid = dict(n_neighbors=n_range, weights=w_options)
#print (param_grid)
model = neighbors.KNeighborsRegressor()

grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

### Final reports for KNN 

In [24]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-28130.858517892786
{'n_neighbors': 5, 'weights': 'distance'}
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='distance')
mean square error 30012.6684618907
root mean square error 173.24164759632916
mean absolute error 94.75166362077671


## SVM 

In [27]:
# Optimize the SVM model by hypertuning using cross validation
# Different parameters for tuning the model

param_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100],'epsilon':[0.1,0.2,0.3]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000],'epsilon':[0.1,0.2,0.3]}]

#print (param_grid)
model = SVR()

#mean_squared_error
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1, 10, 100], 'epsilon': [0.1, 0.2, 0.3],
                          'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'epsilon': [0.1, 0.2, 0.3],
                          'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [28]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-18527.505585533283
{'C': 1000, 'epsilon': 0.3, 'kernel': 'linear'}
SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.3,
    gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
mean square error 17736.974098941395
root mean square error 133.18023163721182
mean absolute error 71.81484107663087


## Random Forest

In [34]:
#Finding the best parameters
tuned_parameters = {"n_estimators" : [10, 100, 200], "max_depth" : [10],
                     "min_samples_leaf" : [20, 40],"max_features" :['sqrt']}
#print (param_grid)
model = RandomForestRegressor() 

grid = GridSearchCV(model, tuned_parameters, cv = 10, scoring = "neg_mean_squared_error")
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10], 'max_featur

In [35]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)


-22177.62615154665
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 20, 'n_estimators': 200}
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=20, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
mean square error 20471.435726542393
root mean square error 143.0784250910751
mean absolute error 83.69684854800131


## Gradient Boosting 

In [36]:
param_grid = {'learning_rate': [0.05,0.1,0.15], 'n_estimators': [100,200],
                     'max_depth':[8],'max_features':['sqrt']
                     ,'subsample':[0.8]}

In [37]:
#Finding the best hyperparameters
model = ensemble.GradientBoostingRegressor()

#mean_squared_error
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_ite...
                             

In [38]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-16981.635112278287
{'learning_rate': 0.05, 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 100, 'subsample': 0.8}
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='ls', max_depth=8,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
mean square error 18898.868634090304
root mean square error 137.47315604906402
mean absolute error 77.16649537458545


## Neural nets

In [82]:
# define base model
def baseline_model():
    model = Sequential()
    model.add(Dense(22, input_dim=22, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

#Data to train the model

X = x_train
Y = y_train

# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=50, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -30033.62 (10619.65) MSE


In [84]:
#Increasing one layer

# define the model
def larger_model():
    model = Sequential()
    model.add(Dense(22, input_dim=22, kernel_initializer='normal', activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -16305.27 (6310.66) MSE


In [85]:
#Increasing one more layer of the model
        
# define the model
def larger_model():
    model = Sequential()
    model.add(Dense(22, input_dim=22, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=25, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -15664.11 (6195.87) MSE


# question 1 part 2

# Running the different regressors on the dataset where purchase is equal to 1

In [63]:
#split dataset in features and target variable
event.loc[:,"Purchase"]==1
event1=event[event.loc[:,"Purchase"]==1]
X=event1.copy(deep=True)
columns = ['sequence_number', 'Purchase', 'Spending']
X.drop(columns, inplace=True, axis=1)
y = event1.loc[:,'Spending'] # Target variable

#Test_train_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=42)

In [65]:
#Pre-processing - scaling of data 
scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(X_train)
x_train = pd.DataFrame(x_train_scaled)

x_test_scaled = scaler.fit_transform(X_test)
x_test = pd.DataFrame(x_test_scaled)

## Regression Tree

In [41]:
# Optimize the Decision tree  by hyperparameter tuning using cross validation
# Defining different parameters for tuning the model
d_range = list(range(1,20))
c_options = ["mse", "mae"]
samples_split=list(np.arange(0.05, 0.5, 0.05))
leaf_nodes=[10]

param_grid = dict(max_depth = d_range, criterion = c_options,min_samples_split=samples_split,max_leaf_nodes=leaf_nodes)
#print (param_grid)
model = tree.DecisionTreeRegressor()

grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19],
           

### Final reports for the Decision tree 

In [42]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-29695.78672669974
{'criterion': 'mse', 'max_depth': 4, 'max_leaf_nodes': 10, 'min_samples_split': 0.1}
DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
                      max_leaf_nodes=10, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.1, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
mean square error 40279.47518028626
root mean square error 200.6974717834937
mean absolute error 117.18188559926098


## Lasso regression 

In [45]:
# Different parameters for tuning the model
#Linear model
param_grid = {'alpha':list(np.arange(0.1,2,0.1))}
#print (param_grid)
model=linear_model.Lasso()

#mean_squared_error
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6,
                                   0.7000000000000001, 0.8, 0.9, 1.0, 1.1,
                                   1.2000000000000002, 1.3000000000000003,
                                   1.4000000000000001, 1.5000000000000002, 1.6,
                                   1.7000000000000002, 1.8000000000000003,
                                   1.9000000000000001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

### Final reports for lasso regression

In [46]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-25818.26161734962
{'alpha': 1.3000000000000003}
Lasso(alpha=1.3000000000000003, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
mean square error 30543.970722000166
root mean square error 174.76833443733497
mean absolute error 105.68420142712681


## Ridge regression 

In [48]:
# Different parameters for tuning the model
param_grid = {'alpha':list(np.arange(0.1,2,0.1))}
model=linear_model.Ridge()
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6,
                                   0.7000000000000001, 0.8, 0.9, 1.0, 1.1,
                                   1.2000000000000002, 1.3000000000000003,
                                   1.4000000000000001, 1.5000000000000002, 1.6,
                                   1.7000000000000002, 1.8000000000000003,
                                   1.9000000000000001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

### Final reports for Ridge regression

In [49]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-26414.037310966454
{'alpha': 0.30000000000000004}
Ridge(alpha=0.30000000000000004, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
mean square error 30136.20631105547
root mean square error 173.5978292233387
mean absolute error 106.52013725234761


## Linear regression without regularisation

In [43]:
# Different parameters for tuning the model
param_grid = {}
#print (param_grid)
model = LinearRegression()

#mean_squared_error
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None, param_grid={}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

### Final reports for linear regression

In [44]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-26454.665773513087
{}
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
mean square error 30009.67073896936
root mean square error 173.23299552616805
mean absolute error 106.67706446199857


## KNN 

In [50]:
# Optimize the KNN model by hypertuning using cross validation
# Different parameters for tuning the model
n_range = list(range(1,30))
w_options = ["uniform", "distance"]

param_grid = dict(n_neighbors=n_range, weights=w_options)
#print (param_grid)
model = neighbors.KNeighborsRegressor()

grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

### Final reports for KNN 

In [51]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-38301.50348802166
{'n_neighbors': 29, 'weights': 'distance'}
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=29, p=2,
                    weights='distance')
mean square error 54857.05130528657
root mean square error 234.2158220643656
mean absolute error 129.739443143942


## SVM 

In [66]:
# Optimize the SVM model by hypertuning using cross validation
# Different parameters for tuning the model

param_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100],'epsilon':[0.1,0.2,0.3]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000],'epsilon':[0.1,0.2,0.3]}]

model = SVR()

#mean_squared_error
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1, 10, 100], 'epsilon': [0.1, 0.2, 0.3],
                          'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'epsilon': [0.1, 0.2, 0.3],
                          'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [67]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-28457.568299695242
{'C': 1000, 'epsilon': 0.2, 'kernel': 'linear'}
SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.2,
    gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
mean square error 34704.58646183507
root mean square error 186.29167040379198
mean absolute error 104.38973704543547


## Gradient Boosting 

In [70]:
param_grid = {'learning_rate': [0.05,0.1,0.15], 'n_estimators': [100,200],
                     'max_depth':[8],'max_features':['sqrt']
                     ,'subsample':[0.8]}

In [71]:
#Finding the best hyperparameters
model = ensemble.GradientBoostingRegressor()

#mean_squared_error
grid = GridSearchCV(model, param_grid, cv = 10,scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_ite...
                             

In [72]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)

-25743.97584754156
{'learning_rate': 0.05, 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 100, 'subsample': 0.8}
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='ls', max_depth=8,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
mean square error 35716.28403099431
root mean square error 188.987523479711
mean absolute error 110.2712582806939


## Random Forest

In [68]:
#Finding the best parameters
tuned_parameters = {"n_estimators" : [10, 100, 200], "max_depth" : [10],
                     "min_samples_leaf" : [20, 40],"max_features" :['sqrt']}
#print (param_grid)
model = RandomForestRegressor() 

grid = GridSearchCV(model, tuned_parameters, cv = 10, scoring = "neg_mean_squared_error")
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10], 'max_featur

In [69]:
#print(grid.grid_scores_)
#print(grid.grid_scores_[0].parameters)
#print(grid.grid_scores_[0].cv_validation_scores)
#print(grid.grid_scores_[0].mean_validation_score)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

#final fitted model
bclf1 = grid.best_estimator_
bclf1.fit(x_train,y_train)
pred = bclf1.predict(x_test)

mse=mean_squared_error(y_test, pred)
print("mean square error",mse)
rms = sqrt(mean_squared_error(y_test, pred))
print("root mean square error",rms)
mae=mean_absolute_error(y_test, pred)
print("mean absolute error",mae)


-32695.80681231426
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 20, 'n_estimators': 100}
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=20, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
mean square error 46115.042246286524
root mean square error 214.74413204156832
mean absolute error 120.83272881083086


## Neural nets

In [75]:
# define base model
def baseline_model():
    model = Sequential()
    model.add(Dense(22, input_dim=22, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

#data to train the model

X = x_train
Y = y_train

# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=50, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -45299.82 (13134.41) MSE


In [76]:
#Increasing one layer

# define the model
def larger_model():
    model = Sequential()
    model.add(Dense(22, input_dim=22, kernel_initializer='normal', activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model 
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -27978.68 (13048.21) MSE


In [78]:
#Increasing one more layer of the model
        
# define the model
def larger_model():
    model = Sequential()
    model.add(Dense(22, input_dim=22, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=25, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -25469.20 (10967.99) MSE
